# Report on results:
With the SGD, we receive worse results than GD as expected but SGD runs much faster and can handle much larger datasets.

With GD, we can deal with 500.000 data points but it takes A LOT of time to fit.

With SGD batch size 1, we can deal with over 1.500.000 data points but the results are not really satisfactory.

With SGD batch size 1000, we can deal with 1.500.000 data points and the results are beter than SGD batch size 1

Also, with SGD, the loss and error function maps are quite unpredictable compared to the constantly decreasing function received from GD. That is because with SGD, it is possible that in each iteration we randomly select a bad sample that causes divergence. Which will lead to ups and downs in the loss and RMSE graph.

In [ ]:
import numpy as np
import time
import random
import scipy.sparse as sps
from scipy.sparse.linalg import svds
import scipy.linalg as lalg
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
def binning(x, bns):
    for b in bns:
        if((x >= b[0]) & (x <= b[1])):
            return b[2]
    return 0

In [ ]:
def loadData(fname, B, N):
    i = 0
    data = []
    uid = -1
    una = ''
    
    bins = [[np.power(2,i), np.power(2,(i + 1)) - 1, i + 1] for i in range(B)]
    bins[len(bins) - 1][1] = np.infty
    
    with open(fname) as FileObj:
        for line in FileObj:
            if(i >= N):
                break
            a = line.split('\t')

            if(a[0] != una):
                uid += 1
                una = a[0]

            data.append([uid, int(a[1],36), binning(int(a[2]),bins)])
            i+=1
    data = np.array(data)
    data = data[data[:,1].argsort()]
    sid = -1
    sti = 0

    for i in range(len(data)):
        if(sti != data[i][1]):
            sti = data[i][1]
            sid += 1
        data[i][1] = sid

    M = sps.coo_matrix((data[:,2],(data[:,0],data[:,1])), dtype=np.float).tocsr()
    return M

In [ ]:
def refineData(M,cutoff):
    carryOn = True

    while(carryOn):

        carryOn = False
        NZ = M.nonzero()

        sList, sCount = np.unique(NZ[1],return_counts=True)
        keepSongs = np.sort(sList[sCount > cutoff])

        if(len(keepSongs) < len(sList)):
            carryOn = True
            M = M[:,keepSongs]

        uList, uCount = np.unique(NZ[0],return_counts=True)
        keepUsers = np.sort(uList[uCount > cutoff])

        if(len(keepUsers) < len(uList)):
            carryOn = True
            M = M[keepUsers,:]
    return M

In [ ]:
def divDS(M, divS):
    NZ = M.nonzero()
    ss = random.sample(range(len(NZ[0])), divS)
    
    valSet = []
    for s in ss:
        i = NZ[0][s]
        j = NZ[1][s]
        valSet.append([i,j,M[i,j]])
        M[i,j] = 0
    
    return valSet,M
    

In [ ]:
def gradientDescent(M,Q,P,valSet,lRate,lambda1,lambda2,maxIter=50):
    print("Applying regular gradient descent")
    print("\t\tTraining RMSE\tValidation RMSE")
    NZ = M.nonzero()
    NZS = len(NZ[0])

    val_e = []
    tr_e = []
    val_l = []
    tr_l = []
    V_s = len(valSet)
    lve = np.infty

    for itr in range(1, maxIter+1):

        val_p = -1
        tra_p = -1

        Q_p = Q.copy()
        P_p = P.copy()

        Q_e = np.zeros(Q.shape)
        P_e = np.zeros(P.shape)

        error = 0
        p1 = 0
        p2 = 0
        
        for nz in range(NZS):
            i = NZ[0][nz]
            x = NZ[1][nz]

            e = M[i,x] - np.dot(Q[:,i], P[:,x])

            p_e = -e * Q[:,i] + lambda1 * P[:,x]
            q_e = -e * P[:,x] + lambda2 * Q[:,i]

            Q_e[:,i] -= q_e * lRate
            P_e[:,x] -= p_e * lRate
            
            error += e*e
            p1 += np.sum(Q[:,NZ[0][nz]] * Q[:,NZ[0][nz]])
            p2 += np.sum(P[:,NZ[1][nz]] * P[:,NZ[1][nz]])

        Q = Q + Q_e
        P = P + P_e

        tr_e.append(np.sqrt(error/NZS))
        tr_l.append((error + lambda1 * p1 + lambda2 * p2) / NZS)

        print("iteration", itr,":  ", np.sqrt(error/NZS),end="\t")

        error = 0
        p1 = 0
        p2 = 0

        for n in range(V_s):
            e = valSet[n][2] - np.dot(Q[:,valSet[n][0]], P[:,valSet[n][1]])
            error += e*e
            p1 += np.sum(Q[:,NZ[0][nz]] * Q[:,NZ[0][nz]])
            p2 += np.sum(P[:,NZ[1][nz]] * P[:,NZ[1][nz]])

        val_e.append(np.sqrt(error/V_s))
        val_l.append((error + lambda1 * p1 + lambda2 * p2) / V_s)

        print(np.sqrt(error/V_s))

        if(lve < np.sqrt(error/V_s)):
            break
        lve = np.sqrt(error/V_s)

    print("Regular gradient descent ended in",itr,"iterations")
    
    if(itr == maxIter + 1):
        return Q,P,val_e,tr_e,val_l,tr_l

    val_e.pop()
    tr_e.pop()
    
    val_l.pop()
    tr_l.pop()
    
    return Q_p,P_p,val_e,tr_e,val_l,tr_l

In [ ]:
def stochasticGradientDescent(M,Q,P,valSet,lRate,lambda1,lambda2,batchSize=1,maxIter=500):
    print("Applying stochastic gradient descent with batch size", batchSize)
    print("\t\tTraining RMSE\tValidation RMSE")
    NZ = M.nonzero()
    NZS = len(NZ[0])

    val_e = []
    tr_e = []
    val_l = []
    tr_l = []
    V_s = len(valSet)
    lve = np.infty
    
    sList, sCount = np.unique(NZ[0],return_counts=True)
    uList, uCount = np.unique(NZ[1],return_counts=True)
    
    TR = random.sample(range(NZS), NZS)

    for itr in range(0, maxIter):

        val_p = -1
        tra_p = -1

        Q_p = Q.copy()
        P_p = P.copy()

        Q_e = np.zeros(Q.shape)
        P_e = np.zeros(P.shape)

        trv = TR[(itr*batchSize)%NZS:((itr+1)*batchSize)%NZS]
        
        error = 0
        p1 = 0
        p2 = 0
        
        for nz in range(len(trv)):
            i = NZ[0][trv[nz]]
            x = NZ[1][trv[nz]]
            
            e = M[i,x] - np.dot(Q[:,i], P[:,x])

            p_e = -e * Q[:,i] + lambda1 * P[:,x]
            q_e = -e * P[:,x] + lambda2 * Q[:,i]

            Q_e[:,i] -= q_e * lRate * uCount[i] / NZS
            P_e[:,x] -= p_e * lRate * sCount[i] / NZS
            
            error += e*e
            p1 += np.sum(Q[:,NZ[0][nz]] * Q[:,NZ[0][nz]])
            p2 += np.sum(P[:,NZ[1][nz]] * P[:,NZ[1][nz]])
           
        tr_e.append(np.sqrt(error/NZS))
        tr_l.append((error + lambda1 * p1 + lambda2 * p2) / NZS)
        
        Q = Q + Q_e
        P = P + P_e
    
        print("iteration", itr,":  ", np.sqrt(error/NZS),end="\t")
        error = 0
        p1 = 0
        p2 = 0

        for n in range(V_s):
            e = valSet[n][2] - np.dot(Q[:,valSet[n][0]], P[:,valSet[n][1]])
            error += e*e
            p1 += np.sum(Q[:,NZ[0][nz]] * Q[:,NZ[0][nz]])
            p2 += np.sum(P[:,NZ[1][nz]] * P[:,NZ[1][nz]])

        val_e.append(np.sqrt(error/V_s))
        val_l.append((error + lambda1 * p1 + lambda2 * p2) / V_s)        
        
        print(np.sqrt(error/V_s))
            
        lve = np.sqrt(error/V_s)

    print("Stochastic gradient descent ended in",itr,"iterations")
    
    if(itr == maxIter - 1):
        return Q,P,val_e,tr_e,val_l,tr_l

    val_e.pop()
    tr_e.pop()
    
    val_l.pop()
    tr_l.pop()
    
    return Q_p,P_p,val_e,tr_e,val_l,tr_l

### LOAD DATA, REFINE QP DECOMPOSITION ETC...

In [ ]:
fname = "train_triplets.txt"
N = 300000
V_s = 1000
T_s = 200
cutoff = 5
B = 10
K = 30

In [ ]:
M = loadData(fname, B, N)
M = refineData(M, cutoff)
valSet, M = divDS(M, V_s)
testSet, M = divDS(M, T_s)

U, s, V = svds(M, k=K)
S = lalg.sqrtm(np.diag(s))
Q = U.dot(S)
Q = Q.transpose()
P = S.dot(V)
Q_bck = Q.copy()
P_bck = P.copy()

lRate = 0.005
lambda2 = 0.02
lambda1 = 0.02

### Standard GD

In [ ]:
begin = time.clock()
Q,P,val_e,tr_e,val_l,tr_l = gradientDescent(M,Q,P,valSet,lRate,lambda1,lambda2)
print("Regular gradient descent took", time.clock() - begin, "seconds") 

In [ ]:
error = 0
for n in range(T_s):
    e = testSet[n][2] - np.dot(Q[:,testSet[n][0]], P[:,testSet[n][1]])
    error += e*e
error = np.sqrt(error/T_s)
print("Testing RMSE:",error)

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_l)),tr_l, 'r')
plt.plot(range(len(val_l)),val_l, 'b')
plt.xlabel("Loss")
plt.ylabel("Time")
plt.show()

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_e)),tr_e, 'r')
plt.plot(range(len(val_e)),val_e, 'b')
plt.xlabel("RMSE")
plt.ylabel("Time")
plt.show()

### Stochastic Gradient Descent with batch size 1

In [ ]:
Q = Q_bck.copy()
P = P_bck.copy()
begin = time.clock()
Q,P,val_e,tr_e,val_l,tr_l = stochasticGradientDescent(M,Q,P,valSet,lRate,lambda1,lambda2,batchSize=1)
print("Stochastic gradient descent with batch size 1 took", time.clock() - begin, "seconds")

In [ ]:
error = 0
for n in range(T_s):
    e = testSet[n][2] - np.dot(Q[:,testSet[n][0]], P[:,testSet[n][1]])
    error += e*e
error = np.sqrt(error/T_s)
print("Testing RMSE:",error)

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_l)),tr_l, 'r')
plt.plot(range(len(val_l)),val_l, 'b')
plt.xlabel("Loss")
plt.ylabel("Time")
plt.show()

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_e)),tr_e, 'r')
plt.plot(range(len(val_e)),val_e, 'b')
plt.xlabel("RMSE")
plt.ylabel("Time")
plt.show()

### Stochastic Gradient Descent with batch size 1000

In [ ]:
Q = Q_bck.copy()
P = P_bck.copy()

begin = time.clock()
Q,P,val_e,tr_e,val_l,tr_l = stochasticGradientDescent(M,Q,P,valSet,lRate,lambda1,lambda2,batchSize=1000)
print("Stochastic gradient descent with batch size 1 took", time.clock() - begin, "seconds")

In [ ]:
error = 0
for n in range(T_s):
    e = testSet[n][2] - np.dot(Q[:,testSet[n][0]], P[:,testSet[n][1]])
    error += e*e
error = np.sqrt(error/T_s)
print("Testing RMSE:",error)

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_l)),tr_l, 'r')
plt.plot(range(len(val_l)),val_l, 'b')
plt.xlabel("Loss")
plt.ylabel("Time")
plt.show()

red_patch = mpatches.Patch(color='red', label='Training')
blue_patch = mpatches.Patch(color='blue', label='Validation')
plt.legend(handles=[red_patch,blue_patch])
plt.plot(range(len(tr_e)),tr_e, 'r')
plt.plot(range(len(val_e)),val_e, 'b')
plt.xlabel("RMSE")
plt.ylabel("Time")
plt.show()